In [46]:
import tweepy
import json
from requests_oauthlib import OAuth1Session
import time
import lib
from notion.client import NotionClient
import pandas as pd 

%load_ext autoreload
%autoreload 2


def notion_to_pandas():
    # Obtain the `token_v2` value by inspecting your browser cookies on a logged-in (non-guest) session on Notion.so
    client = NotionClient(token_v2="8f3ff0a1f8ee1758ce8d1ad19741cff2dac2197f520c382b76dd3652723f68e8d889d717ea8c0f134d0da8fc18497a6abfc071e1e93242e7a181768d100c419cd6704def7cf59c22f65944504943")
    url="https://www.notion.so/ede84dc00e8c42c1bcff78751313c82e?v=3ad69769c83148f5aaf02d1313387f1a"

    # Replace this URL with the URL of the page you want to edit
    #page = client.get_block("https://www.notion.so/myorg/Test-c0d20a71c0944985ae96e661ccc99821")

    #print("The old title is:", page.title)
    res=client.get_collection_view(url)
    tdf=[]
    col=res.collection
    for r in col.get_rows():
        try:
            tdf.append(r.get_all_properties())
        except:
            pass
    
    tdf=pd.DataFrame(tdf)
    #tdf=pd.DataFrame(col.columns[0])
    
    #tdf.columns=col
    import re
    for i,row in tdf.iterrows():        
        text=re.sub("\[|\]", "",row.tweet)
        tdf.tweet[i]=re.sub("\(.+?\)", "", text)

    return tdf

tdf=notion_to_pandas()
api=lib.getApiInstance()

#res=client.get_collection_view(url)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<ipython-input-46-062df13efcc1>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf.tweet[i]=re.sub("\(.+?\)", "", text)


In [52]:

import tweepy
#r=list(tweepy.Cursor(api.search_users, q ="社会福祉士", lang = 'ja').items(10))

#api=lib.getApiInstance()
#screen_name="evidencefukusi"
#user=api.get_user(screen_name="evidencefukusi")
client = NotionClient(token_v2="8f3ff0a1f8ee1758ce8d1ad19741cff2dac2197f520c382b76dd3652723f68e8d889d717ea8c0f134d0da8fc18497a6abfc071e1e93242e7a181768d100c419cd6704def7cf59c22f65944504943")
url="https://www.notion.so/ede84dc00e8c42c1bcff78751313c82e?v=3ad69769c83148f5aaf02d1313387f1a"

# Replace this URL with the URL of the page you want to edit
#page = client.get_block("https://www.notion.so/myorg/Test-c0d20a71c0944985ae96e661ccc99821")

#print("The old title is:", page.title)
#res=client.get_collection_view(url)
#help(res.collection)

#res=list(tweepy.Cursor(api.search, q ="おっぱい",  include_entities = True, tweet_mode = 'extended', lang = 'ja').items(10))



In [ ]:
import numpy as np
import re
import sqlite3

def search(search_query):
    global Ntweet,api
    print(Ntweet)
    tweet=tweepy.Cursor(api.search, q =search_query,  include_entities = True, tweet_mode = 'extended', lang = 'ja').items(Ntweet)    
    #tweet=api.search(search_query,count=5000)
    texts=[]
    profile=[]

    df=[]
    for r in tweet:        
        #print(r)
        df.append(r)    
        texts.append(r.user.name+r.user.description+"  "+r.full_text)
        #df.append({"name":r.user.name,"location":r.user.location,"description":r.user.description})        

    #print(len(texts))
    return df,texts


def run_bot(dat):
    while True:
        d=tdf.iloc[np.random.choice(tdf.shape[0],1)[0]]
        key=d.search_word
        
        #tweepy.Cursor(api.search_users, q ="社会福祉士", lang = 'ja').items(100) 
        #if d.tweettype!="tweet":
        #    continue
        if d.tweettype=="tweet":
            tweets=d["tweet"]
            #print(np.random.choice(tweets,1)[0])
            try:
                api.update_status(np.random.choice(tweets,1)[0])
            except:
                pass
            time.sleep(600)
            continue            
        elif d.tweettype=="search":            
            try:
                tweets,texts=search(key)
            except:
                time.sleep(ST)
                continue
        elif d.tweettype=="search_profile":
            try:
                tweets,texts=search(key)
            except:
                time.sleep(ST)
                continue
        else:
            continue
            
        #print(tweets)
        print(d)
        retweetWithComment(d,tweets,
                           texts=texts)        
        time.sleep(ST)

def insertusertable(r):
    u=r.user
    dd=[u.id,u.screen_name,u.name,search_query,u.description,r.full_text,u.location]    
    cur.execute("insert into users(id,screen_name,name,search_query,description,tweet,location) values(?,?,?,?,?,?,?)",dd)
    
def inserttweet(tw):
    tw=[r.id,r.full_text,r.user.screen_name,r.user.name,r.user.description,search_query,r.created_at,r.user.location,r.retweet_count]
    cur.execute("insert into tweets(id,tweet,screen_name,name,description,search_query,created_at,location,retweet_count) values(?,?,?,?,?,?,?,?,?)",tw)
    

def retweetWithComment(d,tweets,texts=[]):
    global gtest,cur
    
    search_query=d.search_word
    cnt=0
    for i in range(len(tweets)):
        k=np.random.choice(range(len(tweets)),1)[0]
        tw=tweets[k]
            
        print("minret",d.min_retweet,d["min_retweet"])
        try:
            if tw.retweet_count<=d["min_retweet"]:
                continue
        except:
            pass

        u=tw.user
        #print(u)
        urls="https://twitter.com/"+u.screen_name+"/status/"+str(tw.id)    
        
        if len(u.location)>0:            
            locstr=u.location.replace("日本","")
            locstr=locstr[:12]
            locstr=locstr+"の"
        else:
            locstr=""
        
        try:
            insertusertable(u)
            #cur.execute("insert into users(screen_name,name,description,location) values(?,?,?,?)",dd)            
            conn.commit()            
        except:
            pass
        try:
            inserttweet(tw)
        except:
            pass
            
        if len(d.condition_profile)>0 and not(d.condition_profile in u.description):
            continue            
            
        
        tweet=d["tweet"]
        '''
        try:
            tweet=np.random.choice(d["tweet"],1)[0]
        except:
            tweet=d["tweet"]
        '''
        print(d["pickup"],type(d["pickup"]),bool(["pickup"]))
        if bool(d["pickup"]):     
            nname=15
            if "士" in search_query:
                ptweet=locstr+u.name[:15]+" @"+u.screen_name[:15]+"さん "+tweet
            else:
                ptweet="#"+search_query+" を利用中の"+locstr+u.name[:nname]+" @"+u.screen_name[:nname]+"さん "+tweet
        else:
            ptweet=tweet            
        m=re.search("https?://[\w/:%#\$&\?\(\)~\.=\+\-]+",ptweet)
        #l=len(m.group())
        if m.start()+11>140:
            ptweet=ptweet[:m.start()]
        print(ptweet)
        print("strlength",len(ptweet))
        
        ptweet+=" "+urls        
        
        
        try:
            if gtest:
                print(1)
            else:          
                print("bpost")
                api.update_status(ptweet)
                print("posted")
            cnt+=1
        except:
            continue
            
        return
        '''
        try:
            s=d["ntry"]
        except:
            return
        
        print("NNNNoret",cnt,gtest,i,int(d["ntry"]))
        #input()
        
        if cnt>=int(d["ntry"]):
            return True
                
        time.sleep(int(ST*0.1))
        '''


        
gtest=False

Ntweet=50
if gtest:
    ST=1000
    Ntweet=int(50)
else:
    ST=3600
    Ntweet=int(1000)


dat=tdf
#discord https://discord.com/invite/HpNBWw7KYt
dbname = 'fukusi.db'
conn = sqlite3.connect(dbname)
cur= conn.cursor()
#cur.execute(
#    'CREATE TABLE users(id INTEGER PRIMARY KEY AUTOINCREMENT,        
#     screen_name STRING unique)')

run_bot(dat)
conn.close()
            

1000
min_retweet                                                          0
tweet                LINEで体温の報告を受けたり,アンパンマンを見せるだけで1.5万円も出る茶番事業ですね、税...
tweettype                                                       search
weight                                                                
condition_profile                                                     
taisho                                                                
pickup                                                             1.0
date                                                              None
search_word                                                      放課後デイ
id                                                                    
Name: 7, dtype: object
minret 0 0
1.0 <class 'numpy.float64'> True
#放課後デイ を利用中の見えない障害から始まる多のきよ@障害を持った子を世界一楽 @kiyohitottehitoさん LINEで体温の報告を受けたり,アンパンマンを見せるだけで1.5万円も出る茶番事業ですね、税金の無駄です。利用を控えてください https://bit.ly/2OSOlWV　#放課後デイ #福祉ビジネス
strlength 161
bpost
minret 0 0
1.0 <class 'numpy.float64

In [187]:
#from markdown import markdown
ptweet=tdf.tweet[3]
ptweet="aaasdfdgasdsa4bt89yaprt8apuiv65ciito87voyvyvuliuyviluyvliyuviluyvsudifyuaiyfyafuyabsfdasdfhavleuhtlvaaetvesvneuioansdifbnasdifnbasizduetbaetrbaer"+ptweet
m=re.search("https?://[\w/:%#\$&\?\(\)~\.=\+\-]+",ptweet)
l=len(m.group())
if m.start()+11>140:
    ptweet=ptweet[:m.start()]
    
ptweet

#tdf.tweet[3]


'aaasdfdgasdsa4bt89yaprt8apuiv65ciito87voyvyvuliuyviluyvliyuviluyvsudifyuaiyfyafuyabsfdasdfhavleuhtlvaaetvesvneuioansdifbnasdifnbasizduetbaetrbaerそれは税金の無駄ですね掃除や家事の報告で１万円の報酬が出る茶番事業の利用は控えてください '

In [38]:
import sqlite3
#create table users(screen_name text unique,name text,location text,profile text,shogaimei text,type text,age intager);

dbname = 'fukusi.db'
conn = sqlite3.connect(dbname)
#cur.execute(
#    'CREATE TABLE users(id INTEGER PRIMARY KEY AUTOINCREMENT,        
#     screen_name STRING unique)')
cur.execute("insert into users()
conn.commit()
conn.close()


In [98]:

#df_n=pd.DataFrame(columns=list(map(lambda x:x["name"],col.get_schema_properties())))
#tdf=notion_to_pandas()


    
cm="それは税金の無駄ですね #A型事業所 #B型事業所 #就労移行支援 などの悪質事業の利用は控えてくださいマトモな事に税金が使われなくなり障害者の迷惑になります。自宅で自習や内職をしましょう"
settai="厚労省と福祉の業者団体でも絶対こういうのありますね、全国社会就労センター協議会,きょうされん,全国就労移行支援事業所連絡協議会全国就業センターとか　"+"https://fukusiprob.blogspot.com/2021/03/blog-post_8.html #接待 #福祉ビジネス"

stop='''【悪質な事業所は許せません】
ここの事業所はどうなんだろう？アンケートご協力お願いします(匿名可) → http://bit.ly/3vIZ9HW 
    STOP！税金の無駄・不正 #就労移行支援　#A型事業所　#放課後デイ　#福祉ビジネス'''

dat={
    "放課後デイサービス":{"tweet":["LINEで体温の報告を受けたり,アンパンマンを見せるだけで1.5万円も出る茶番事業ですね、税金の無駄です。利用を控えてください https://bit.ly/2OSOlWV　#放課後デイ #福祉ビジネス",
                          stop,
                         "税金ジャブジャブ障害福祉サービスの利用は控えましょう　https://bit.ly/2OSOlWV　放課後デイ #福祉ビジネス"],
                 "pickup":True},
    
    "A型事業所":{
        "tweet":[
            "税金の無駄ですね掃除や家事の報告で１万円の報酬が出る茶番事業の利用は控えてくださいマトモな事に税金が使われなくなり障害者の迷惑になりアホになります。自習や内職をしましょう https://bit.ly/3tte9HZ",
            "それは税金の無駄ですね掃除や家事の報告で１万円の報酬が出る茶番事業の利用は控えてください  https://bit.ly/3tte9HZ #B型事業所",
            stop],
        "pickup":True        
    },
    "就労移行支援":{
        "tweet":
            ["エクセルやUdemyの自習で1.5万円/日の報酬が出る茶番事業の利用は控えてくださいマトモな事に税金が使われなくなり障害者の迷惑になります。ハロワの基金訓練などを探しましょうhttps://bit.ly/3bUqaQR",
             "エクセルやUdemyの自習で1.5万円/日の報酬が出る茶番事業の利用は控えてくださいマトモな事に税金が使われなくなり障害者の迷惑になります　https://bit.ly/3bUqaQR ",
             stop
            ],
        "pickup":True 
        
    },
    '''
    "社会福祉士":{
        "tweet":
            ["#社会福祉士 #精神保健福祉士 などによる相談ヤクザ的な行為は税金の無駄です。なにかの役に立っているというのは行政と癒着した業者団体の洗脳で、障害者にとって迷惑ですので仕事は控えてください https://sites.google.com/view/mentalmamoru/",
            ],
        "pickup":True      
    },
    "精神保健福祉士":{
        "tweet":
            ["#社会福祉士 #精神保健福祉士 などによる相談ヤクザ的な行為は税金の無駄です。なにかの役に立っているというのは行政と癒着した業者団体の洗脳で、障害者にとって迷惑ですので仕事は控えてください https://sites.google.com/view/mentalmamoru/",
            ],
        "pickup":True      
    },
    '''
    "接待":{
        "tweet":settai,
        "min_retweet":True,
        "ntry":1
    },
    "通常":{
        "tweet":[
            '''
            当厚労省ではOCがあります。discordのほうが性能が良いですが廃れています
            オープンチャット「障害者系のOCの苦情受けつけ件、就労支援などの福祉を正常にする会」 http://bit.ly/3fbZ6yu
            '''
        ]
    }    
}


False

In [101]:
tdf.iloc[4,]

min_retweet                                                     
tweet          エクセルやUdemyの自習で1.5万円/日の報酬が出る茶番事業の利用は控えてくださいマトモな...
tweettype                                                 search
weight                                                          
taisho                                                          
pickup                                                       1.0
date                                                        None
search_word                                               就労移行支援
id                                                              
Name: 4, dtype: object

In [65]:
#for r in res:
#    print(r)


,tweet,column_2,dui_xiang,pickup,date,jian_suo_wado,id
0,,,,NaN,None,,
1,,,,NaN,None,,
2,,,,NaN,None,,
3,税金の無駄使いだと感じる。掃除や家事の報告で１万円の報酬が出る茶番事業の利用は控えないと、マ...,,,1.0,None,B型事業所,
4,"""それは税金の無駄だと思います。掃除や家事の報告で１万円の報酬が出る茶番事業の利用は控えませ...",,,1.0,None,B型事業所,
5,税金の無駄ですね掃除や家事の報告で１万円の報酬が出る茶番事業の利用は控えてくださいマトモな事...,,,1.0,None,A型事業所,
6,それは税金の無駄ですね掃除や家事の報告で１万円の報酬が出る茶番事業の利用は控えてください [...,,,1.0,None,A型事業所,
7,,,,NaN,None,,
8,エクセルやUdemyの自習で1.5万円/日の報酬が出る茶番事業の利用は控えてくださいマトモな...,,,1.0,None,就労移行支援,
9,エクセルやUdemyの自習で1.5万円/日の報酬が出る茶番事業の利用は控えてくださいマトモな...,,,1.0,None,就労移行支援,


In [47]:
df_n

,tweet,Column 2,対象,pickup,Date,検索ワード,id


In [13]:
stop='''【悪質な事業所は許せません】
ここの事業所はどうなんだろう？アンケートご協力お願いします(匿名可) → http://bit.ly/3vIZ9HW 
    STOP！税金の無駄・不正'''
stop

'【悪質な事業所は許せません】\nここの事業所はどうなんだろう？アンケートご協力お願いします(匿名可) → http://bit.ly/3vIZ9HW \n    STOP！税金の無駄・不正'

In [66]:
#list(set([m.text for m in main]))
def blacket(x):    
    x=[str(sx) if type(sx)==int else "'"+sx+"'" for sx in x ]
    ret='('+",".join(x)+')'
    print(ret)
    return ret

blacket(["test",234,"bb"])

('test',234,'bb')


"('test',234,'bb')"

In [87]:
def get_usertype(keyword,profile,tweet):
    res=[]
    if "事業所" in profile:
        return res.append("業者")
    
    

NameError: name 'u' is not defined

In [92]:
"事業所" in "就労移行支援事業所"

True

In [ ]:

#=
run_bot(["A型事業所","就労移行支援","接待"],
       ["それは税金の無駄ですね掃除や家事の報告で１万円の報酬が出る #A型事業所 #B型事業所などの茶番事業の利用は控えてくださいマトモな事に税金が使われなくなり障害者の迷惑になりアホになります。自習や内職をしましょう",
        "エクセルやUdemyでの自習で1.5万円/日の報酬が出る #就労移行支援 などの茶番事業の利用は控えてくださいマトモな事に税金が使われなくなり障害者の迷惑になります。ハロワの基金訓練などを探しましょう",
        settai
       ],
       [1,1,2],
       [0,0,20],
       [1,1,0])
#=
    

In [93]:
import pandas as pd
dat={
    "放課後デイサービス":{"tweet":"LINEで体温の報告を聞いただけで1万数千円も出るヤツですね、税金の無駄です。利用を控えてください　#放課後デイサービス #福祉ビジネス ",
                 "pickup":True},
    
    "A型事業所":{
        "tweet":"それは税金の無駄ですね掃除や家事の報告で１万円の報酬が出る #A型事業所 #B型事業所などの茶番事業の利用は控えてくださいマトモな事に税金が使われなくなり障害者の迷惑になりアホになります。自習や内職をしましょう",
        "pickup":True        
    },
    "就労移行支援":{
        "tweet":"エクセルやUdemyでの自習で1.5万円/日の報酬が出る #就労移行支援 などの茶番事業の利用は控えてくださいマトモな事に税金が使われなくなり障害者の迷惑になります。ハロワの基金訓練などを探しましょう",
        "pickup":True 
        
    },
    "接待":{
        "tweet":settai                
    }    
}

d=dat["A型事業所"]
list(dat.keys())
"A型事業所" in dat

True

In [ ]:
def retweetWithComment_df(search_query,tweet,df):
    for i in range(len(df)):
        #tw=np.random.choice(df,1)[0]
        tw=df[i]
        print(tw.retweet_count)
        if tw.retweet_count<=10:
            continue
        u=tw.user
        urls="https://twitter.com/"+u.screen_name+"/status/"+str(tw.id)    
        
        if len(u.location)>0:
            locstr=u.location+"の "
            locstr=locstr.replace("日本","")
            locstr=locstr[:12]
        else:
            locstr=""

        #ptweet=search_query+"を利用中の"+locstr+u.name+" @"+u.screen_name[:20]+"さん "+comments[0]+" "
        ptweet=tweet+" "+urls
        ptweet=ptweet[:140]
        print(ptweet)
        print(len(ptweet))
        try:
            api.update_status(ptweet+" "+urls)
        except:
            continue
        
        time.sleep(1800)


retweetWithComment(q,"厚労省と福祉の業者団体でも絶対こういうのありますね、全国社会就労センター協議会,きょうされん,全国就労移行支援事業所連絡協議会全国就業センターとか　"+"https://fukusiprob.blogspot.com/2021/03/blog-post_8.html #接待 #福祉ビジネス",df)




In [61]:
import time
import numpy as np


for i in range(100,len(df)):
    tw=np.random.choice(df,1)[0]
    u=tw.user
    urls="https://twitter.com/"+u.screen_name+"/status/"+str(tw.id)    
    
    if len(u.location)>0:
        locstr=u.location+"の "
        locstr=locstr.replace("日本","")
        locstr=locstr[:12]
    else:
        locstr=""
    
    ptweet=search_query+"を利用中の"+locstr+u.name+" @"+u.screen_name[:20]+"さん "+comments[0]+" "
    ptweet=ptweet[:140]
    print(ptweet)
    print(len(ptweet))
    #api.update_status(ptweet+" "+urls)
    time.sleep(1800)

    

就労移行支援を利用中の就労移行支援Re.born @shuroshienweb2さん それは税金の無駄ですね #A型事業所 #B型事業所 #就労移行支援 などの悪質事業の利用は控えてくださいマトモな事に税金が使われなくなり障害者の迷惑になります。自宅で自習や内職をしましょう 
138


KeyboardInterrupt: 

0
388
厚労省と福祉の業者団体でも絶対こういうのありますね、全国社会就労センター協議会,きょうされん,全国就労移行支援事業所連絡協議会全国就業センターとか　https://fukusiprob.blogspot.com/2021/03/blog-post_8.html #接待 #福祉ビジ
140
0
0
0
2
18
厚労省と福祉の業者団体でも絶対こういうのありますね、全国社会就労センター協議会,きょうされん,全国就労移行支援事業所連絡協議会全国就業センターとか　https://fukusiprob.blogspot.com/2021/03/blog-post_8.html #接待 #福祉ビジ
140
882
厚労省と福祉の業者団体でも絶対こういうのありますね、全国社会就労センター協議会,きょうされん,全国就労移行支援事業所連絡協議会全国就業センターとか　https://fukusiprob.blogspot.com/2021/03/blog-post_8.html #接待 #福祉ビジ
140
2
6
129
厚労省と福祉の業者団体でも絶対こういうのありますね、全国社会就労センター協議会,きょうされん,全国就労移行支援事業所連絡協議会全国就業センターとか　https://fukusiprob.blogspot.com/2021/03/blog-post_8.html #接待 #福祉ビジ
140
372
厚労省と福祉の業者団体でも絶対こういうのありますね、全国社会就労センター協議会,きょうされん,全国就労移行支援事業所連絡協議会全国就業センターとか　https://fukusiprob.blogspot.com/2021/03/blog-post_8.html #接待 #福祉ビジ
140
4
613
厚労省と福祉の業者団体でも絶対こういうのありますね、全国社会就労センター協議会,きょうされん,全国就労移行支援事業所連絡協議会全国就業センターとか　https://fukusiprob.blogspot.com/2021/03/blog-post_8.html #接待 #福祉ビジ
140
731
厚労省と福祉の業者団体でも絶対こういうのありますね、全国社会就労センター協議会,きょうされん,全国就労移行支援事業所連絡協議会全国就業センターとか　https://fukus

In [74]:
tw=np.random.choice(df,1)[0]
u=tw.user
urls="https://twitter.com/"+u.screen_name+"/status/"+str(tw.id)    
if len(u.location)>0:
    ptweet=u.location+"の "+u.name+"(@"+u.screen_name+")さん "+comments[0]+" "+urls
else:
    ptweet=u.name+"(@"+u.screen_name+")さん "+comments[0]+" "+urls

print(ptweet)
api.update_status(ptweet)


香織(@ka04aiai)さん 税金の無駄ですね、 #A型事業所 #B型事業所　#就労移行支援　などの悪質事業の利用は控えてください、使われるとマトモな事に税金が使われなくなり、障害者の迷惑になります。散歩をして自宅で自習や内職をしましょう。 https://twitter.com/ka04aiai/status/1369828297207545856


Status(_api=<tweepy.api.API object at 0x115d27d60>, _json={'created_at': 'Mon Mar 15 08:55:49 +0000 2021', 'id': 1371384631082119168, 'id_str': '1371384631082119168', 'text': '香織(@ka04aiai)さん 税金の無駄ですね、 #A型事業所 #B型事業所\u3000#就労移行支援\u3000などの悪質事業の利用は控えてください、使われるとマトモな事に税金が使われなくなり、障害者の迷惑になります。散歩をして自宅で自習や内職を… https://t.co/Lr0aApt6Ys', 'truncated': True, 'entities': {'hashtags': [{'text': 'A型事業所', 'indices': [26, 32]}, {'text': 'B型事業所', 'indices': [33, 39]}, {'text': '就労移行支援', 'indices': [40, 47]}], 'symbols': [], 'user_mentions': [{'screen_name': 'ka04aiai', 'name': '香織', 'id': 774242154151456768, 'id_str': '774242154151456768', 'indices': [3, 12]}], 'urls': [{'url': 'https://t.co/Lr0aApt6Ys', 'expanded_url': 'https://twitter.com/i/web/status/1371384631082119168', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'source': '<a href="https://script.google.com/macros/d/1GOaodfD17lPyVX0xDoZ02n5dAigWp4M5b96-MZYoEwJiXkx44xs9ykTX/" rel="nofollow">analyze fukushi</a>', 'in_reply

In [50]:
i=90
u=df[i].user
urls="https://twitter.com/"+u.screen_name+"/status/"+str(df[i].id)    
ptweet=comments[0]+" "+urls
print(ptweet)
api.update_status(ptweet)


税金の無駄ですね、利用を控えてください https://twitter.com/KumatanSukiyaki/status/1370285806200389634


Status(_api=<tweepy.api.API object at 0x115d27d60>, _json={'created_at': 'Mon Mar 15 07:27:20 +0000 2021', 'id': 1371362363694215172, 'id_str': '1371362363694215172', 'text': '税金の無駄ですね、利用を控えてください https://t.co/4fsJADqiMd', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/4fsJADqiMd', 'expanded_url': 'https://twitter.com/KumatanSukiyaki/status/1370285806200389634', 'display_url': 'twitter.com/KumatanSukiyak…', 'indices': [20, 43]}]}, 'source': '<a href="https://script.google.com/macros/d/1GOaodfD17lPyVX0xDoZ02n5dAigWp4M5b96-MZYoEwJiXkx44xs9ykTX/" rel="nofollow">analyze fukushi</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1288065283219832834, 'id_str': '1288065283219832834', 'name': '福祉ムラをぶっ壊してエビデンスベースの福祉を実現する会', 'screen_name': 'evidencefukusi', 'location': '市川市', 'description': 'データサイエンス